In [99]:
#pip install transformers
#pip install torch

import requests
from bs4 import BeautifulSoup
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta
import base64
import json

In [100]:
# Definir os temas de interesse
temas = "ciência de dados, inteligência artificial"
limite_resultados = 100
filtro_tempo = "dia"

# Mapeamento de filtros de data para o parâmetro do Google Notícias
filtros_tempo = {
    "dia": "qdr:d",
    "semana": "qdr:w",
    "mês": "qdr:m"
}
filtro_param = filtros_tempo.get(filtro_tempo, "")

# Definir o limite de tempo (7 dias atrás) para filtrar as notícias
limite_dias = 7
data_limite = datetime.now() - timedelta(days=limite_dias)

# Converter os temas em uma lista e armazenar os resultados na lista de dicionários
temas_list = [tema.strip() for tema in temas.split(',')]
news_data = []  # Lista para armazenar os dados de cada notícia

# Loop para buscar notícias de cada tema
for tema in temas_list:
    query = tema.replace(' ', '%20')  # Formatar o tema para URL
    url = f"https://news.google.com/search?q={query}&hl=pt-BR&gl=BR&ceid=BR%3Apt-419&tbs={filtro_param}"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('article')

    # Extrair títulos, links, imagens, sites e datas para cada artigo
    for article in articles:
        if len(news_data) >= limite_resultados:
            break

        title_tag = article.find('a', class_='JtKRv')
        image_tag = article.find('img', class_='Quavad vwBmvb')
        site_name_tag = article.find('div', class_='vr1PYe')
        time_tag = article.find('time')

        if title_tag:
            title = title_tag.text.strip()

            # Construir o link completo
            link = title_tag['href']
            if link.startswith('/'):
                link = f"https://news.google.com{link}"
            elif link.startswith('./'):
                link = f"https://news.google.com{link[1:]}"  # Remove o "." inicial
            elif not link.startswith('http'):
                link = f"https://news.google.com/{link}"

            # Construir o link da imagem
            if image_tag and 'srcset' in image_tag.attrs:
                image_src = image_tag['srcset'].split()[0]
                image_link = f"https://news.google.com{image_src}" if image_src.startswith('/') else image_src
            else:
                image_link = "Imagem não disponível"

            site_name = site_name_tag.text.strip() if site_name_tag else "Nome do site não disponível"

            # Filtrar pela data e ignorar notícias mais antigas que o limite de 7 dias
            if time_tag:
                publication_date = time_tag['datetime']
                noticia_data = datetime.strptime(publication_date, "%Y-%m-%dT%H:%M:%SZ")
                if noticia_data < data_limite:
                    continue  # Ignora esta notícia se for mais antiga que a data limite
            else:
                publication_date = "Data não disponível"

            # Adicionar a notícia à lista de dicionários
            news_data.append({
                'Manchete': title,
                'Link': link,
                'Link_img': image_link,
                'Site': site_name,
                'Data': publication_date,
                'Relevancia': 0
            })

# Convertendo a lista de dicionários para um DataFrame
df = pd.DataFrame(news_data)

In [101]:
# Remover duplicados, mantendo apenas a primeira ocorrência
df = df.drop_duplicates(subset='Manchete', keep='first')

# df[df.duplicated(subset='Manchete', keep=False)] # verifica se existem duplicados

df.head()

,Manchete,Link,Link_img,Site,Data,Relevancia
0,Encerram-se domingo as inscrições no Programa ...,https://news.google.com/read/CBMiwwFBVV95cUxQS...,https://news.google.com/api/attachments/CC8iK0...,AEROIN,2024-11-04T16:48:10Z,0
1,"Transformação de plásticos, ciência de dados, ...",https://news.google.com/read/CBMikwJBVV95cUxPb...,https://news.google.com/api/attachments/CC8iK0...,IFBA,2024-11-01T14:52:00Z,0
2,Contrata-se: demanda por cientistas de dados c...,https://news.google.com/read/CBMirwFBVV95cUxON...,https://news.google.com/api/attachments/CC8iK0...,InfoMoney,2024-10-30T12:15:00Z,0
3,Ciência de Dados e Inteligência Artificial: ab...,https://news.google.com/read/CBMivgFBVV95cUxOY...,https://news.google.com/api/attachments/CC8iK0...,sejabixo.com.br,2024-11-04T22:23:52Z,0
4,10 cursos gratuitos de ciência de dados para i...,https://news.google.com/read/CBMidkFVX3lxTE10U...,https://news.google.com/api/attachments/CC8iK0...,Seu Crédito Digital,2024-11-01T02:20:00Z,0


In [102]:
# Frases de referência para calcular a relevância
frases_referencia = [
    "Inteligência artificial e aprendizado de máquina",
    "Avanços em ciência de dados e big data",
    "Inovações em machine learning e IA"
]

# Carregar o modelo e tokenizer do Hugging Face
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Função para calcular a média da similaridade entre a manchete e múltiplas frases de referência
def calcular_similaridade_com_media(manchete):
    pontuacoes = []
    for frase in frases_referencia:
        inputs_manchete = tokenizer(manchete, return_tensors="pt", truncation=True, max_length=512)
        inputs_frase = tokenizer(frase, return_tensors="pt", truncation=True, max_length=512)

        # Gerar embeddings
        embedding_manchete = model(**inputs_manchete).last_hidden_state.mean(dim=1)
        embedding_frase = model(**inputs_frase).last_hidden_state.mean(dim=1)

        # Calcular similaridade e adicionar à lista de pontuações
        similaridade = cosine_similarity(embedding_manchete.detach().numpy(), embedding_frase.detach().numpy())
        pontuacoes.append(similaridade[0][0])

    # Retornar a média das pontuações de similaridade
    return sum(pontuacoes) / len(pontuacoes)

# Aplicar a função para calcular a relevância de cada manchete no DataFrame
df['Relevancia'] = df['Manchete'].apply(calcular_similaridade_com_media)

# Ordena o DataFrame pela Relevância
df = df.sort_values(by='Relevancia', ascending=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [103]:
# Gera um Json.
# O parâmetro orient='records' organiza os dados como uma lista de dicionários / force_ascii=False manterá os caracteres especiais corretamente codificados

# json = df.to_json('noticias.json', orient='records', force_ascii=False)

# Exibir o DataFrame atualizado com a coluna de relevância
df.head()

,Manchete,Link,Link_img,Site,Data,Relevancia
18,Inteligência Artificial identifica alta da Sel...,https://news.google.com/read/CBMiogFBVV95cUxPW...,Imagem não disponível,VEJA,2024-11-04T18:10:51Z,0.535724
26,Caixa oferece bolsas de estudo em Inteligência...,https://news.google.com/read/CBMib0FVX3lxTE5rR...,https://news.google.com/api/attachments/CC8iI0...,Sindpd,2024-11-05T13:46:27Z,0.525853
39,A crise da leitura: o impacto na era da Inteli...,https://news.google.com/read/CBMixAFBVV95cUxQQ...,https://news.google.com/api/attachments/CC8iK0...,Portal iG,2024-11-05T08:00:00Z,0.513915
3,Ciência de Dados e Inteligência Artificial: ab...,https://news.google.com/read/CBMivgFBVV95cUxOY...,https://news.google.com/api/attachments/CC8iK0...,sejabixo.com.br,2024-11-04T22:23:52Z,0.513046
54,Inteligência artificial aponta quem vence no c...,https://news.google.com/read/CBMitgFBVV95cUxOZ...,https://news.google.com/api/attachments/CC8iL0...,Super Rádio Tupi,2024-11-05T14:19:27Z,0.507345


In [ ]:
# atualiza o git ----------------------

# Geração do JSON como string diretamente do DataFrame
json_content = df.to_json(orient='records', force_ascii=False)

# Codificar o conteúdo JSON em base64
content_encoded = base64.b64encode(json_content.encode('utf-8')).decode('utf-8')

# Configurações do GitHub
token = 'seu-token' # substituir com o token
repo = 'leonelbr/ainews'
path = 'noticias.json'
url_base = f'https://api.github.com/repos/{repo}/contents/{path}'

# Headers para autenticação
headers = {
    'Authorization': f'token {token}',
    'Accept': 'application/vnd.github.v3+json'
}

# Verificar o SHA do arquivo atual (necessário para atualização)
response = requests.get(url_base, headers=headers)

# Depuração para verificar a resposta do GET inicial
print("Resposta GET inicial:", response.status_code, response.json() if response.status_code != 200 else "Arquivo encontrado")

# Capturar o SHA do arquivo existente, se ele já estiver no repositório
if response.status_code == 200:
    sha = response.json()['sha']
    print("SHA atual encontrado:", sha)  # Depuração do SHA
else:
    sha = None  # Arquivo não existe, será criado
    print("SHA não encontrado. O arquivo será criado.")  # Indicação de criação

# Dados para envio ou atualização do arquivo
data = {
    "message": "Atualização do arquivo de notícias",
    "content": content_encoded,
    "sha": sha  # SHA é obrigatório para atualizar o arquivo, mas opcional para criar um novo
}

# Enviar o arquivo atualizado para o GitHub
response = requests.put(url_base, headers=headers, data=json.dumps(data))

# Verificar a resposta final
print("Resposta PUT:", response.status_code)
if response.status_code in [200, 201]:
    print("Arquivo enviado com sucesso.")
    print("URL de confirmação:", response.json().get("content", {}).get("html_url", "URL não disponível"))
else:
    print("Erro ao enviar o arquivo:", response.json())

Resposta GET inicial: 200 Arquivo encontrado
SHA atual encontrado: 4d58d30f818220cd3e3955343edc71726a9073e9
Resposta PUT: 200
Arquivo enviado com sucesso.
URL de confirmação: https://github.com/leonelbr/ainews/blob/main/noticias.json
